In [ ]:
<VSCode.Cell language="markdown"># Import Libraries and Configure Paths

Import necessary libraries and set up file paths for data and outputs.</VSCode.Cell>
<VSCode.Cell language="python">import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Define directories
SCRIPT_DIR = "analysis"
ER_CSV = os.path.join(SCRIPT_DIR, "synthetic_er_visits_by_zip.csv")
OUTPUT_DIR = os.path.join(SCRIPT_DIR, "outputs")
os.makedirs(OUTPUT_DIR, exist_ok=True)
</VSCode.Cell>
<VSCode.Cell language="markdown"># Load CSV Data and Vulnerability Table

Read emergency room visits data and create a DataFrame of heat vulnerability metrics by ZIP code.</VSCode.Cell>
<VSCode.Cell language="python"># Load ER visits data
df = pd.read_csv(ER_CSV, parse_dates=["date"])
df["zip"] = df["zip"].astype(str)
df = df.sort_values(["zip", "date"])

# Vulnerability metrics table
HVI_AC = pd.DataFrame({
    "zip": ["10453","10454","11212","11405","10029"],
    "neighborhood": ["Morrisania","Hunts Point","Brownsville","South Jamaica","East Harlem"],
    "hvi_level": [5,5,5,5,4],
    "pct_without_ac": [24.2,15.7,18.3,15.8,15.1]
})
</VSCode.Cell>
<VSCode.Cell language="markdown"># Compute 7-Day Baseline and Excess Visits

Calculate the 7-day rolling median baseline of ER visits and compute excess visits over this baseline.</VSCode.Cell>
<VSCode.Cell language="python">df["baseline_er"] = (
    df.groupby("zip")["er_visits"]
      .transform(lambda s: s.rolling(7, min_periods=3).median())
)
df = df.dropna(subset=["baseline_er"])
df["excess_er"] = df["er_visits"] - df["baseline_er"]
</VSCode.Cell>
<VSCode.Cell language="markdown"># Calculate 3-Day Rolling Heat Index and Excess

Compute 3-day rolling averages of heat index and excess ER visits.</VSCode.Cell>
<VSCode.Cell language="python">df["hi_3day"] = df.groupby("zip")["max_heat_index"]
                     .transform(lambda s: s.rolling(3, min_periods=1).mean())
df["excess_3day"] = df.groupby("zip")["excess_er"]
                         .transform(lambda s: s.rolling(3, min_periods=1).mean())
</VSCode.Cell>
<VSCode.Cell language="markdown"># Determine HI Trigger Thresholds

Find the first heat index threshold per ZIP where excess visits reach ≥20% of baseline for at least one day.</VSCode.Cell>
<VSCode.Cell language="python">thresholds = []
for z, grp in df.groupby("zip"):
    for hi in np.arange(85, 105, 0.5):
        mask = grp["hi_3day"] >= hi
        if mask.sum() < 3:
            continue
        if (grp.loc[mask, "excess_3day"] >= 0.2 * grp.loc[mask, "baseline_er"]).any():
            thresholds.append({"zip": z, "hi_trigger": hi})
            break
trigger_df = pd.DataFrame(thresholds)
</VSCode.Cell>
<VSCode.Cell language="markdown"># Merge Dataframes and Flag High-Risk Days

Combine datasets and mark days as high risk based on heat index, HVI, and AC vulnerability.</VSCode.Cell>
<VSCode.Cell language="python">df = df.merge(HVI_AC, on="zip", how="left")
df = df.merge(trigger_df, on="zip", how="left")

df["high_risk"] = (
    (df["hi_3day"] >= df["hi_trigger"]) &
    (df["hvi_level"] >= 4) &
    (df["pct_without_ac"] >= 20)
)
</VSCode.Cell>
<VSCode.Cell language="markdown"># Create Summary Table of Trigger Days

Aggregate the number of trigger days per ZIP and save the summary to CSV.</VSCode.Cell>
<VSCode.Cell language="python">summary = (
    df[df["high_risk"]]
    .groupby(["zip","neighborhood","hvi_level","pct_without_ac","hi_trigger"])
    .size()
    .reset_index(name="num_trigger_days")
)
summary.to_csv(os.path.join(OUTPUT_DIR, "trigger_summary_table.csv"), index=False)
summary
</VSCode.Cell>
<VSCode.Cell language="markdown"># Plot Enhanced Bar Chart: Trigger Days by Neighborhood

Styled bar plot with annotations and custom palette.</VSCode.Cell>
<VSCode.Cell language="python">plt.figure(figsize=(10,6))
sns.barplot(
    data=summary,
    x="neighborhood",
    y="num_trigger_days",
    palette="Reds_d"
)
plt.title("Trigger Days by Neighborhood", fontsize=14)
plt.xlabel("Neighborhood", fontsize=12)
plt.ylabel("Number of Trigger Days", fontsize=12)
plt.xticks(rotation=45, ha="right")
for index, row in summary.iterrows():
    plt.text(index, row.num_trigger_days + 0.5, int(row.num_trigger_days), ha="center")
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "enhanced_bar_trigger_days.png"))
plt.show()
</VSCode.Cell>
<VSCode.Cell language="markdown"># Plot Enhanced Bar Chart: HI Trigger Threshold by Neighborhood

Bar plot showing calibrated heat index thresholds with consistent formatting.</VSCode.Cell>
<VSCode.Cell language="python">plt.figure(figsize=(10,6))
sns.barplot(
    data=summary,
    x="neighborhood",
    y="hi_trigger",
    palette="Blues_d"
)
plt.title("HI Trigger Threshold by Neighborhood", fontsize=14)
plt.xlabel("Neighborhood", fontsize=12)
plt.ylabel("HI Trigger (°F)", fontsize=12)
plt.xticks(rotation=45, ha="right")
for index, row in summary.iterrows():
    plt.text(index, row.hi_trigger + 0.2, f"{row.hi_trigger:.1f}", ha="center")
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "enhanced_bar_hi_trigger.png"))
plt.show()
</VSCode.Cell>